In [1]:
import pandas as pd
import re
import numpy as np
import requests
import ckanapi
import json
import os 

# Obetener tabla id + nombre

#### Localhost

In [23]:
ckan =ckanapi.RemoteCKAN('http://0.0.0.0', apikey=APIKEY)
datasets=ckan.action.current_package_list_with_resources()

#### Opendatatest

In [8]:
ckan =ckanapi.RemoteCKAN('http://opendata-test.ugr.es', apikey=APIKEY)




#### Opendata

In [2]:

ckan =ckanapi.RemoteCKAN('https://opendata.ugr.es', apikey=APIKEY)
datasets=ckan.action.package_list()

In [3]:
len(datasets)

52

In [4]:
idnombre=pd.DataFrame(columns=['dataset','id','nombre'])
datasets=ckan.action.package_list()
for dataset in datasets:
        

    resources=ckan.action.package_show(id=dataset)
    for resource in resources['resources']:
        #print (resource['name'])
        idnombre=idnombre.append({'dataset':dataset, 'id':resource['id'],'nombre':resource['name']},ignore_index=True)
        
        


In [5]:
idnombre

,dataset,id,nombre
0,academico,b47349d8-a3fc-4bff-972a-56f45b1c020a,Tasas académicas por titulaciones 2015
1,academico,99ae9ef6-3131-48b7-9c76-3926b78bb196,Tasas académicas por titulaciones 2014
2,academico,06fcfcca-e4d2-484f-8c07-f8aa6ca35eb1,Tasas académicas por ramas 2013/2014
3,academico,9fa254dc-08ae-4649-a497-f71754f4ac4b,Número medio de créditos 2013/2014
4,academico,9afc8a94-619f-4a46-9e69-46ad8753d520,Tasas académicas por titulaciones 2013
...,...,...,...
437,titulaciones-14-15,4feb3c28-36c1-4a22-b76b-2a48be46ace6,Oferta de posgrado: doctorado
438,titulaciones-2015-2016,02d7d7a3-e78c-498a-93b9-beda768e2d80,Oferta de grado
439,titulaciones-2015-2016,4b1fb8b7-f3c0-41ef-8f76-c91b85063df8,Oferta de posgrado: másteres oficiales
440,titulaciones-2015-2016,94ee412c-c4e0-4034-a87b-c85c7f3f306e,Oferta de posgrado: doctorado


# Cargar el log de Nginx

In [5]:
log=pd.read_table('/home/juanma/Escritorio/prueba25octEstadisticas/access.log.1', names=['info'])
#log=pd.read_table('/var/log/nginx/access.log', names=['info'])
log['info']

0        185.191.171.17 - - [24/Oct/2022:00:00:07 +0200...
1        185.191.171.13 - - [24/Oct/2022:00:00:07 +0200...
2        157.55.39.46 - - [24/Oct/2022:00:00:09 +0200] ...
3        150.214.21.81 - - [24/Oct/2022:00:00:17 +0200]...
4        185.191.171.12 - - [24/Oct/2022:00:00:17 +0200...
                               ...                        
30437    185.191.171.11 - - [24/Oct/2022:23:59:54 +0200...
30438    40.77.167.9 - - [24/Oct/2022:23:59:55 +0200] "...
30439    185.191.171.9 - - [24/Oct/2022:23:59:56 +0200]...
30440    157.55.39.46 - - [24/Oct/2022:23:59:59 +0200] ...
30441    114.119.155.210 - - [25/Oct/2022:00:00:01 +020...
Name: info, Length: 30442, dtype: object

# Filtrar log con descargas

In [6]:
exIP = re.compile(r'(\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})') 
exFecha = re.compile(r'(\d{1,2}\/\[a-z]\/\d{4})')
logFiltrado=pd.DataFrame(columns=['IP', 'Fecha','Accion'])
#lista=[]
for line in log['info']:
    
    
    l=line.split(sep=' ')
    
    #Descargas desde boton
    #200 5334 -> vista a resource 200 1186 -> vista a datastore
    if ('GET /' in line and ('/download/' in line or '/dump/' in line)):
        #lista.append(exIP.match(line).group())
        ip=exIP.match(line).group()
        
        #Eliminar primer [
        fecha=l[3][1:]
        pos=fecha.find(':')
        fecha=fecha[:pos]

        #Accion
        accion=l[5][1:] +' ' +l[6]
        logFiltrado=logFiltrado.append({'IP': ip, 'Fecha': fecha, 'Accion':accion}, ignore_index=True)
        
    #elif ('GET /dataset/' in line and '/resource' in line and '/view' not in line and '/dataset/ HTTP' not in line and '/resource/new' not in line and 'http://opendata-test.ugr.es/dataset' not in line):
    elif ('GET /dataset/' in line and '/resource' in line and '/view' not in line and '/dataset/ HTTP' not in line and '/resource/new' not in line ):
        ip=exIP.match(line).group()
        #Eliminar primer [
        fecha=l[3][1:]
        pos=fecha.find(':')
        fecha=fecha[:pos]
        #accion
        accion=l[5][1:] +' ' +l[6]
        cadena= accion.split(sep='/')
        ##MEJORAR LA BUSQUEDA DE VISTAS -----> log de recargar vista de resource
      
        if ((len(cadena)) > 3):
            if (cadena[3] == 'resource'):
                logFiltrado=logFiltrado.append({'IP': ip, 'Fecha': fecha, 'Accion':accion}, ignore_index=True)
    
    
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth',3000)
logFiltrado

,IP,Fecha,Accion
0,114.119.155.154,24/Oct/2022,GET /dataset/36dc4e7f-678e-48cb-b901-5541c410e043/resource/f92b1419-45c6-43af-b9ba-c93ef71995bf
1,54.36.149.65,24/Oct/2022,GET /dataset/desglose-ingresos-2014/resource/17645a87-0cfb-4d4f-b415-c16ccbd7433c?view_id=f13ff4ed-061a-41b5-aa38-04a458107110
2,185.191.171.1,24/Oct/2022,GET /dataset/memorias-de-investigacion-2014/resource/1ed47ceb-47e8-4007-b34f-304acba6fb17
3,114.119.155.126,24/Oct/2022,GET /sl/dataset/6bafcf73-06a4-421a-8b2f-b7ce6ce6e757/resource/a7892d77-4d83-4d84-9fca-7625e8688b2b/download/masteryotros.csv
4,114.119.155.145,24/Oct/2022,GET /dataset/memoria-de-investiacion-2017/resource/776911ad-54fb-4e4f-b4a9-26122174fe38?inner_span=True
...,...,...,...
702,85.52.202.157,24/Oct/2022,GET /dataset/3a597af1-43f8-442e-8783-940f5161e52f/resource/0ebb2b54-2445-436a-bed3-6d70add9a8a0/download/proyectosprovisonales2016.xlsx
703,85.52.202.157,24/Oct/2022,GET /dataset/3a597af1-43f8-442e-8783-940f5161e52f/resource/0ebb2b54-2445-436a-bed3-6d70add9a8a0/download/proyectosprovisonales2016.xlsx
704,185.191.171.3,24/Oct/2022,GET /dataset/719545d3-e3f4-4eb6-b54b-07967a5e74e4/resource/ebace326-794d-4921-9702-5f306831bb2b/preview
705,54.36.148.221,24/Oct/2022,GET /dataset/livemetrics/resource/5d0b7301-48b7-47bc-8aae-4c2eb0f72e58?view_id=f035eefc-ff91-4e79-8eee-f94d65cbd6cb


# Recopilación del nº descargas junto a resource

In [7]:



existe = os.path.exists('stats.json')

if (existe):
    print ("Cargando archivo stats.json")
    descargas = pd.read_json('stats.json')
    resourcesD=descargas['resource'].tolist()

else:
    print ("Creando archivo stats.json")
    descargas= pd.DataFrame(columns=['dataset','resource','nombre','descargas','visitas'])
    resourcesD=descargas['resource'].tolist()

dump=0
click=0
visita=0

for acc in logFiltrado['Accion']:
    cadena= acc.split(sep='/')
    
    
    if ('/dump/' in acc):
        
        c=cadena[3]
        #En caso de se que hagan consultas desde otros idiomas
        if (c == 'dump'):
            c=cadena[4]
        
        #Eliminamos el ?format=....
        pos=c.find('?')
        if (pos != -1):
            c=c[:pos]
        
        
        
        
        #Aumento contador
        if (c in resourcesD):
            dump=dump+1
            index=np.where(descargas['resource'] == c)[0][0]
            descargas.at[index,'descargas']=descargas.at[index,'descargas'] + 1
            
        #Añado uno nuevo
        elif (c in idnombre['id'].tolist()):
            
            dump=dump+1
            index=np.where(idnombre['id'] == c)[0][0]
            nombre=idnombre.at[index,'nombre']
            dataset=idnombre.at[index,'dataset']
            descargas=descargas.append({'dataset':dataset,'resource':c,'descargas':1,'nombre':nombre,'visitas':0}, ignore_index=True)
            resourcesD=descargas['resource'].tolist()
        
        #print ("Accion desde dump : " + str(acc))

    elif ('/download/' in acc):
        
        if (len(cadena) >=5):
            c=cadena[4]
        else:
            continue;
        
        if (c == 'resource'):
            c=cadena[5]
        
        pos=c.find('?')
        if (pos != -1):
            c=c[:pos]
        
        
        if (c in resourcesD):
            click=click+1
            index=np.where(descargas['resource'] == c)[0][0]
            descargas.at[index,'descargas']=descargas.at[index,'descargas'] + 1
        elif (c in idnombre['id'].tolist()):
            
            click=click+1
            index=np.where(idnombre['id'] == c)[0][0]
            nombre=idnombre.at[index,'nombre']
            dataset=idnombre.at[index,'dataset']
                
            descargas=descargas.append({'dataset':dataset,'resource':c,'descargas':1,'visitas':0,'nombre':nombre}, ignore_index=True)
            resourcesD=descargas['resource'].tolist()
            

    elif ('GET /dataset/' in acc and '/resource' in acc and '/view' not in acc and '/dataset/ HTTP' not in acc):
        
        if (len(cadena) >= 5):
            c=cadena[4].split(sep=' ')[0]
            
        else:
            continue;
        
        pos=c.find('?')
        if (pos != -1):
            c=c[:pos]
        
        
        if (c in resourcesD):
            visita=visita+1
            index=np.where(descargas['resource'] == c)[0][0]
            descargas.at[index,'visitas']=descargas.at[index,'visitas'] + 1
            

        elif (c in idnombre['id'].tolist()):
            visita=visita+1
            index=np.where(idnombre['id'] == c)[0][0]


            nombre=idnombre.at[index,'nombre']
            dataset=idnombre.at[index,'dataset']

            descargas=descargas.append({'dataset':dataset,'resource':c,'descargas':0,'nombre':nombre,'visitas':1}, ignore_index=True)
            resourcesD=descargas['resource'].tolist()
            
        
        
print ('Desdse dump --> ' + str(dump))
print ('Desde click --> ' + str(click))
print ('Visitas realizadas --> ' + str(visita))

statsjson = descargas.to_json(index=True)
statsjson = json.loads(statsjson)
with open('stats.json','w' ) as json_file:
    json.dump(statsjson,json_file)


Cargando archivo stats.json
Desdse dump --> 181
Desde click --> 307
Visitas realizadas --> 191


In [8]:
estadisticas = pd.read_json('stats.json')
estadisticas

,dataset,resource,nombre,descargas,visitas
0,estadisticas-del-portal-de-datos-abiertos,5293c58e-53ad-4628-9306-b969b6939bb5,statsFormatos.json,6,38
1,prueba-de-publicacion-twitter,c7d35cd5-b092-4474-978e-bb7b3a2fd4cd,Indicadores Académicos UGR,3,35
2,presupuesto-2014,0511325c-cea7-44fc-af3e-40eae0ea9e17,Actuaciones programa 321B: estructura y gestión universitaria,4,6
3,memoria-de-investiacion-2017,d6d775d9-fa6d-4465-8d60-57b85fc1293b,9. Ranking de investigadores global de la UGR según el número de citas en Google Scholar,1,6
4,desglose-ingresos-2014,e66940ec-13a8-4e89-a3ac-6ba530492a69,"Detalle por subconcepto, cap. 4",9,4
5,ficheros-informes-bibliometricos,0ebb2b54-2445-436a-bed3-6d70add9a8a0,Proyectos Retos y Excelencia - 2016,22,5
6,presupuesto-2018,926c31e4-f376-4580-8703-3a75d4786d88,Actuaciones programa 321B: estructura y gestión universitaria,0,12
7,titulaciones-13-14,e544794c-0763-4447-9d3c-17effb2726c0,Oferta de posgrado: formación del profesorado,4,9
8,livemetrics,3174d09a-ded3-44b9-90ef-587b4da9a0e1,Colaboración Institucional - ac,9,2
9,investigacion,a39b500a-67fd-4bbd-9ce4-a5731154964c,Proyectos obtenidos: Programa Estatal (2013),5,4


In [9]:
print ("Visitas sum: ",estadisticas['visitas'].sum())
print ("Descargas sum: ",estadisticas['descargas'].sum())

Visitas sum:  1583
Descargas sum:  1525


# Recopilación descargas / visitas por mes

In [10]:

existe = os.path.exists('statsDescargasVisitasMesAño.json')

if (existe):
    print ("Cargando archivo statsDescargasVisitasMesAño.json")
    descargasVisitasMes = pd.read_json('statsDescargasVisitasMesAño.json')
    

else:
    print ("Creando archivo statsDescargasVisitasMesAño.json")
    descargasVisitasMes=pd.DataFrame(columns=['mes/año','descargas','visitas'])


descargas=0
visitas=0

fechasGuardadas=descargasVisitasMes['mes/año'].tolist()

for acc in logFiltrado['Accion']:

    
    cadena= acc.split(sep='/')
    #Descarga click
    if ('/download/' in acc):
        
        if (len(cadena)>=5):
            c=cadena[4]
        else:
            continue;
        
        if (c == 'resource'):
            c=cadena[5]
        
        pos=c.find('?')
        if (pos != -1):
            c=c[:pos]
        
        
        if (c in idnombre['id'].tolist()):
            descargas=descargas+1
        #else:
        #    print ("Descarga ------>" + str(acc) + " " + str(descargas))   

   
    
    elif('/dump/' in acc):
        
        c=cadena[3]
        #En caso de se que hagan consultas desde otros idiomas
        if (c == 'dump'):
            c=cadena[4]
        
        #Eliminamos el ?format=....
        pos=c.find('?')
        if (pos != -1):
            c=c[:pos]
    
        
        if (c in idnombre['id'].tolist()):
            
            descargas=descargas+1
    
    #visita
    elif ('GET /dataset/' in acc and '/resource' in acc and '/view' not in acc and '/dataset/ HTTP' not in acc):
    
        if (len(cadena) >= 5):
            c=cadena[4].split(sep=' ')[0]
            
        else:
            print ("aqui")
            continue;
        
        
        pos=c.find('?')
        if (pos != -1):
            c=c[:pos]
        

        if (c in idnombre['id'].tolist()):
            visitas=visitas+1
        
        
        
        
        
if (logFiltrado.empty == False ):   
    fecha=logFiltrado['Fecha'][0].split(sep='/')[1] +'/'+logFiltrado['Fecha'][0].split(sep='/')[2]
    #mes=logFiltrado['Fecha'][0].split(sep='/')[1]
    #año=logFiltrado['Fecha'][0].split(sep='/')[2]
    #fecha=mes+'/'+año



    if (fecha in fechasGuardadas):
        index=np.where(descargasVisitasMes['mes/año'] == fecha)[0][0]
        descargasVisitasMes.at[index,'descargas']=descargasVisitasMes.at[index,'descargas'] + descargas
        descargasVisitasMes.at[index,'visitas']=descargasVisitasMes.at[index,'visitas'] + visitas
    else:
        descargasVisitasMes=descargasVisitasMes.append({'mes/año':fecha,'descargas':descargas,'visitas':visitas}, ignore_index=True)
    

print ("Descargas: " + str(descargas) + " " + "visitas: " + str(visitas))


statsjson = descargasVisitasMes.to_json(index=True)
statsjson = json.loads(statsjson)
with open('statsDescargasVisitasMesAño.json','w' ) as json_file:
    json.dump(statsjson,json_file)

Cargando archivo statsDescargasVisitasMesAño.json
Descargas: 488 visitas: 191


In [11]:
descargasVisitasMes

,mes/año,descargas,visitas
0,Apr/2022,700,711
1,Oct/2022,813,375


# Número de conjuntos y resources por fecha

In [12]:
from datetime import datetime


fechaActual=datetime.today().strftime('%Y-%m-%d')


existe = os.path.exists('numeroTotalDatasets.json')

if (existe):
    
    numeroDR= pd.read_json('numeroTotalDatasets.json')
    
else:
    numeroDR=pd.DataFrame(columns=['fecha','nº conjuntos'])

    


datasets=ckan.action.package_list()
tamDatasets=len(datasets)-1 #portal eu
tamResources=0

for dataset in datasets:    
    resources=ckan.action.package_show(id=dataset)
    tamResources=len(resources['resources']) + tamResources
    
    
print (tamDatasets) 
print (tamResources)
    


if (fechaActual not in numeroDR['fecha'].tolist() ):

    numeroDR=numeroDR.append({'fecha':fechaActual,'nº conjuntos':tamDatasets,'nº recursos':tamResources}, ignore_index=True)

    statsjson=numeroDR.to_json(index=True)
    statsjson=json.loads(statsjson)
    with open('numeroTotalDatasets.json','w') as json_file:
        json.dump(statsjson,json_file)



51
442


In [13]:
tamDatasets= pd.read_json('numeroTotalDatasets.json')
tamDatasets

,fecha,nº conjuntos,nº recursos
0,2022-05-05,50,441
1,2022-10-25,51,442


# Acceso de IPs

In [14]:

existe = os.path.exists('infoIPs.json')

if (existe):
    print ("Cargando archivo infoIPs.json")
    registroIPs = pd.read_json('infoIPs.json')
    

else:
    print ("Creando archivo infoIPs.json")
    registroIPs= pd.DataFrame(columns=['ip','lat','long','city','state'])



#lista de ip [ip1, ip2, ip3]

listaIPRegistradas=registroIPs['ip'].tolist()
#listaIPRegistradas.append('172.25.22.67')
for i in logFiltrado['IP']:
    
    ip=(i.split(sep=' ')[0])
    
    
    
    if ip not in listaIPRegistradas :
        urlConsulta= 'https://geolocation-db.com/jsonp/' + ip
        response = requests.get(urlConsulta)
        datos=response.content.decode()
        datos=datos.split("(")[1].strip(")")
        datos=json.loads(datos)
        listaIPRegistradas.append(ip)
        if (datos['latitude'] != 'Not found' and datos['longitude'] != 'Not found'):
            registroIPs=registroIPs.append({'ip':ip,'lat':datos['latitude'],'long':datos['longitude'],'city':datos['city'],'state':datos['state'],}, ignore_index=True)
    
    
    
statsIP=registroIPs.to_json(index=True)
statsIP=json.loads(statsIP)
with open('infoIPs.json','w') as json_file:
    json.dump(statsIP,json_file)


Cargando archivo infoIPs.json


# Top formatos utilizados

In [16]:

print ("Creando archivo statsFormatos.json")
formatosD=pd.DataFrame(columns=['formato','contador'])




datasets=ckan.action.package_list()
for dataset in datasets:
        
    #print (dataset)
    datos=ckan.action.package_show(id=dataset)
    for dato in datos['resources']:
        formato= dato['format']
        
        
        if (formato in formatosD['formato'].tolist()):
        
            index=np.where(formatosD['formato'] == formato)[0][0]
            formatosD.at[index,'contador']=formatosD.at[index,'contador'] + 1
            
        else:
            formatosD=formatosD.append({'formato':formato, 'contador': 1 }, ignore_index=True)

            


formatosD=formatosD.sort_values('contador', ascending=False)

statsFormatos = formatosD.to_json(index=True)
statsFormatos = json.loads(statsFormatos)
with open('statsFormatos.json','w' ) as json_file:
    json.dump(statsFormatos,json_file)
    
    
formatosD

Creando archivo statsFormatos.json


,formato,contador
0,CSV,375
4,XLSX,37
3,XLS,20
5,PDF,5
2,JSON,4
1,RDF,1


# Eliminar datos inexistentes

In [19]:

existe = os.path.exists('stats.json')

if (existe):
    print ("Cargando archivo stats.json")
    descargas = pd.read_json('stats.json')
    resourcesD=descargas['resource'].tolist()

    
    #Get index of 'Créditos matriculados'
    resourceName='Créditos matriculados'
    index=np.where(descargas['nombre'] == resourceName)[0][0]
    descargas1=descargas.drop(index=index)
    
descargas1

Cargando archivo stats.json


,dataset,resource,nombre,descargas,visitas
0,profesores-extranjeros,05ec9a34-c63b-4952-8288-dfc659091e3f,Origen del profesorado 2015/2016,1,1
1,criterios-de-la-fundacion-compromiso-y-transpa...,cb461464-8e48-441d-bcd2-0daf5a9a67c9,Criterios de la Fundación Compromiso y Transpa...,1,2
2,presupuesto-2015,bf77c0c2-a04a-4b93-a2fa-e4f865affa89,Créditos distribuibles a centros,0,2
3,rankings-de-investigadores-ugrinvestiga-segun-...,f1fcece5-ff30-4264-9638-cdbb7fd78954,Actualización de los “ Rankings de investigado...,0,1
4,presupuesto-ugr-2013-estado-de-ingresos,ec5e458d-99b3-4970-a01f-e4f8fc51afe9,Presupuesto 2013: desglose de ingresos UGR,1,1
5,presupuesto-2015,8cc06dda-2705-4daf-aeb4-029e6ba7cc7c,Bases de ejecución presupuesto ejercicio 2015,0,1
6,presupuesto-2015,bd4ea51b-a354-448f-897b-3bca214a938c,Créditos distribuibles a centros: detraer,0,1
7,presupuesto-2015,788adaa5-eb7d-4353-94c2-12d9b9af3bb3,Actuaciones 422D: enseñanzas universitarias,0,1
8,presupuesto-2015,b1df3548-9e28-4874-8d2f-c52cc5ca3915,Sección 3ª: actividad académica,2,1
10,bandas-salariales-2015,2a5a0614-155b-47c3-9c8b-d1aa0c077765,"Tramos docentes e investigadores, cargos acadé...",1,1


In [16]:
descargas1.reset_index(drop=True)

,dataset,resource,nombre,descargas,visitas
0,profesores-extranjeros,05ec9a34-c63b-4952-8288-dfc659091e3f,Origen del profesorado 2015/2016,1,1
1,criterios-de-la-fundacion-compromiso-y-transpa...,cb461464-8e48-441d-bcd2-0daf5a9a67c9,Criterios de la Fundación Compromiso y Transpa...,1,2
2,presupuesto-2015,bf77c0c2-a04a-4b93-a2fa-e4f865affa89,Créditos distribuibles a centros,0,2
3,rankings-de-investigadores-ugrinvestiga-segun-...,f1fcece5-ff30-4264-9638-cdbb7fd78954,Actualización de los “ Rankings de investigado...,0,1
4,presupuesto-ugr-2013-estado-de-ingresos,ec5e458d-99b3-4970-a01f-e4f8fc51afe9,Presupuesto 2013: desglose de ingresos UGR,1,1
5,presupuesto-2015,8cc06dda-2705-4daf-aeb4-029e6ba7cc7c,Bases de ejecución presupuesto ejercicio 2015,0,1
6,presupuesto-2015,bd4ea51b-a354-448f-897b-3bca214a938c,Créditos distribuibles a centros: detraer,0,1
7,presupuesto-2015,788adaa5-eb7d-4353-94c2-12d9b9af3bb3,Actuaciones 422D: enseñanzas universitarias,0,1
8,presupuesto-2015,b1df3548-9e28-4874-8d2f-c52cc5ca3915,Sección 3ª: actividad académica,2,1
9,bandas-salariales-2015,2a5a0614-155b-47c3-9c8b-d1aa0c077765,"Tramos docentes e investigadores, cargos acadé...",1,1


In [29]:
index=np.where(descargas['nombre'] == 'hola')[0]
if (index.size == 0):
    print ("hola no ")

True

# Cargar stats.json para sumatoria

In [48]:


print ("Cargando archivo stats.json")
stats = pd.read_json('/home/juanma/Escritorio/LOGNGINXOPENDATAFINAL/stats.json')
stats

Cargando archivo stats.json


,dataset,resource,nombre,descargas,visitas
0,profesores-extranjeros,05ec9a34-c63b-4952-8288-dfc659091e3f,Origen del profesorado 2015/2016,9,6
1,criterios-de-la-fundacion-compromiso-y-transparencia,cb461464-8e48-441d-bcd2-0daf5a9a67c9,Criterios de la Fundación Compromiso y Transparencia,27,32
2,presupuesto-2015,bf77c0c2-a04a-4b93-a2fa-e4f865affa89,Créditos distribuibles a centros,12,34
3,rankings-de-investigadores-ugrinvestiga-segun-citacion,f1fcece5-ff30-4264-9638-cdbb7fd78954,"Actualización de los “ Rankings de investigadores UGRinvestiga según citación""",2,9
4,presupuesto-ugr-2013-estado-de-ingresos,ec5e458d-99b3-4970-a01f-e4f8fc51afe9,Presupuesto 2013: desglose de ingresos UGR,3,9
5,presupuesto-2015,8cc06dda-2705-4daf-aeb4-029e6ba7cc7c,Bases de ejecución presupuesto ejercicio 2015,13,28
6,presupuesto-2015,bd4ea51b-a354-448f-897b-3bca214a938c,Créditos distribuibles a centros: detraer,7,17
7,presupuesto-2015,788adaa5-eb7d-4353-94c2-12d9b9af3bb3,Actuaciones 422D: enseñanzas universitarias,9,16
8,presupuesto-2015,b1df3548-9e28-4874-8d2f-c52cc5ca3915,Sección 3ª: actividad académica,11,20
9,presupuesto-2015,a5855f02-d82c-45a2-b1ab-d9e465e1a0e2,Créditos matriculados,10,24


In [49]:
print ("Visitas sum: ",stats['visitas'].sum())
print ("Descargas sum: ",stats['descargas'].sum())

Visitas sum:  4226
Descargas sum:  1783
